In [2]:
import pandas as pd

movies = pd.read_csv('/Users/berkaybakac/Downloads/movie.csv')
ratings = pd.read_csv('/Users/berkaybakac/Downloads/rating.csv')


In [3]:
# Eksik verileri kontrol etme
print(movies.isnull().sum())
print(ratings.isnull().sum())


movieId    0
title      0
genres     0
dtype: int64
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [4]:
# İlk birkaç satırı görüntüleyerek veri setine göz atma
print(movies.head())
print(ratings.head())

# Veri setindeki sütunların türlerini ve genel bilgileri görüntüleme
print(movies.info())
print(ratings.info())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Colu

In [5]:
# Sayısal sütunların temel istatistiksel analizini yapma
print(ratings.describe())


             userId       movieId        rating
count  2.000026e+07  2.000026e+07  2.000026e+07
mean   6.904587e+04  9.041567e+03  3.525529e+00
std    4.003863e+04  1.978948e+04  1.051989e+00
min    1.000000e+00  1.000000e+00  5.000000e-01
25%    3.439500e+04  9.020000e+02  3.000000e+00
50%    6.914100e+04  2.167000e+03  3.500000e+00
75%    1.036370e+05  4.770000e+03  4.000000e+00
max    1.384930e+05  1.312620e+05  5.000000e+00


In [6]:
# Kaç benzersiz film ve kullanıcı olduğunu hesaplama
unique_movies = movies['movieId'].nunique()
unique_users = ratings['userId'].nunique()

print(f"Benzersiz film sayısı: {unique_movies}")
print(f"Benzersiz kullanıcı sayısı: {unique_users}")


Benzersiz film sayısı: 27278
Benzersiz kullanıcı sayısı: 138493


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vektörleştirici oluşturma (Tür bazlı)
tfidf = TfidfVectorizer(stop_words='english')

# 'genres' sütunundaki veriyi kullanarak TF-IDF matrisini oluşturma
tfidf_matrix = tfidf.fit_transform(movies['genres'])

print(f"TF-IDF matris boyutu: {tfidf_matrix.shape}")


TF-IDF matris boyutu: (27278, 23)


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

# Cosine similarity kullanarak benzerlik matrisini oluşturma
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Örneğin, ilk filmle en benzer filmleri bulma
similar_movies = list(enumerate(cosine_sim[0]))
similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

# İlk filmle en benzer 10 filmi görüntüleme
print("İlk filmle en benzer 10 film:")
for i in similar_movies[1:11]:
    print(f"Film ID: {movies.iloc[i[0]]['movieId']}, Başlık: {movies.iloc[i[0]]['title']}, Benzerlik: {i[1]}")


İlk filmle en benzer 10 film:
Film ID: 2294, Başlık: Antz (1998), Benzerlik: 1.0000000000000002
Film ID: 3114, Başlık: Toy Story 2 (1999), Benzerlik: 1.0000000000000002
Film ID: 3754, Başlık: Adventures of Rocky and Bullwinkle, The (2000), Benzerlik: 1.0000000000000002
Film ID: 4016, Başlık: Emperor's New Groove, The (2000), Benzerlik: 1.0000000000000002
Film ID: 4886, Başlık: Monsters, Inc. (2001), Benzerlik: 1.0000000000000002
Film ID: 33463, Başlık: DuckTales: The Movie - Treasure of the Lost Lamp (1990), Benzerlik: 1.0000000000000002
Film ID: 45074, Başlık: Wild, The (2006), Benzerlik: 1.0000000000000002
Film ID: 53121, Başlık: Shrek the Third (2007), Benzerlik: 1.0000000000000002
Film ID: 65577, Başlık: Tale of Despereaux, The (2008), Benzerlik: 1.0000000000000002
Film ID: 91355, Başlık: Asterix and the Vikings (Astérix et les Vikings) (2006), Benzerlik: 1.0000000000000002


In [9]:
def get_recommendations(title, movies, cosine_sim):
    # Filmin indeksini bulma
    idx = movies[movies['title'].str.contains(title, case=False, na=False)].index[0]
    
    # Filmin benzerlik puanlarını içeren listeyi alma
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Benzerlik puanlarına göre filmleri sırala
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # İlk filmi (kendisi) atla ve en benzer 10 filmi al
    sim_scores = sim_scores[1:11]
    
    # Bu filmlerin indekslerini al
    movie_indices = [i[0] for i in sim_scores]
    
    # Bu filmlerin başlıklarını döndür
    return movies.iloc[movie_indices][['title', 'genres']]

# Örneğin, "Toy Story" filmi için öneriler alalım
recommendations = get_recommendations("Toy Story", movies, cosine_sim)
print(recommendations)


                                                   title  \
2209                                         Antz (1998)   
3027                                  Toy Story 2 (1999)   
3663      Adventures of Rocky and Bullwinkle, The (2000)   
3922                    Emperor's New Groove, The (2000)   
4790                               Monsters, Inc. (2001)   
10114  DuckTales: The Movie - Treasure of the Lost La...   
10987                                   Wild, The (2006)   
11871                             Shrek the Third (2007)   
13337                     Tale of Despereaux, The (2008)   
18274  Asterix and the Vikings (Astérix et les Viking...   

                                            genres  
2209   Adventure|Animation|Children|Comedy|Fantasy  
3027   Adventure|Animation|Children|Comedy|Fantasy  
3663   Adventure|Animation|Children|Comedy|Fantasy  
3922   Adventure|Animation|Children|Comedy|Fantasy  
4790   Adventure|Animation|Children|Comedy|Fantasy  
10114  Adventure|Anim

In [10]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Puanlar veri setini Surprise kütüphanesi için hazırlama
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Veri setini eğitim ve test seti olarak ayırma
trainset, testset = train_test_split(data, test_size=0.2)

# SVD modeli oluşturma ve eğitme
model = SVD()
model.fit(trainset)

# Test setinde tahminler yapma
predictions = model.test(testset)

# Modelin performansını değerlendirme (RMSE)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 0.7862
RMSE: 0.7862384298267552


In [11]:
# SVD modeli oluşturma ve eğitme (verbose=True ile eğitim durumunu izleyebilirsiniz)
model = SVD(verbose=True)
model.fit(trainset)


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [12]:
# Puanlar veri setinden küçük bir alt küme oluşturma
ratings_small = ratings.sample(100000, random_state=42)  # Belirli bir alt küme alıyoruz (100.000 satır)

# Puanlar veri setini Surprise kütüphanesi için hazırlama
data = Dataset.load_from_df(ratings_small[['userId', 'movieId', 'rating']], reader)

# Veri setini eğitim ve test seti olarak ayırma
trainset, testset = train_test_split(data, test_size=0.2)

# SVD modeli oluşturma ve eğitme
model = SVD()
model.fit(trainset)

# Test setinde tahminler yapma
predictions = model.test(testset)

# Modelin performansını değerlendirme (RMSE)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 0.9582
RMSE: 0.9582332410330945


In [13]:
def get_user_recommendations(user_id, model, movies, ratings, num_recommendations=10):
    # Kullanıcının daha önce puan verdiği filmleri alalım
    watched_movies = ratings[ratings['userId'] == user_id]['movieId'].tolist()
    
    # Kullanıcının henüz puanlamadığı filmleri bulma
    all_movies = movies['movieId'].tolist()
    unseen_movies = [movie for movie in all_movies if movie not in watched_movies]
    
    # Henüz puanlanmamış filmler için tahmin yapma
    predictions = [model.predict(user_id, movie_id) for movie_id in unseen_movies]
    
    # Tahminleri puanlara göre sırala
    predictions = sorted(predictions, key=lambda x: x.est, reverse=True)
    
    # En iyi tahmin edilen filmleri al ve başlıklarını döndür
    top_predictions = predictions[:num_recommendations]
    recommended_movie_ids = [pred.iid for pred in top_predictions]
    
    return movies[movies['movieId'].isin(recommended_movie_ids)][['title', 'genres']]

# Örneğin, kullanıcı ID 1 için öneriler alalım
user_recommendations = get_user_recommendations(user_id=1, model=model, movies=movies, ratings=ratings_small)
print(user_recommendations)


                                                   title  \
49                            Usual Suspects, The (1995)   
257            Star Wars: Episode IV - A New Hope (1977)   
315                     Shawshank Redemption, The (1994)   
523                              Schindler's List (1993)   
737    Dr. Strangelove or: How I Learned to Stop Worr...   
843                                Godfather, The (1972)   
1149      Cinema Paradiso (Nuovo cinema Paradiso) (1989)   
1171   Star Wars: Episode V - The Empire Strikes Back...   
4877   Amelie (Fabuleux destin d'Amélie Poulain, Le) ...   
15534                                   Inception (2010)   

                                                genres  
49                              Crime|Mystery|Thriller  
257                            Action|Adventure|Sci-Fi  
315                                        Crime|Drama  
523                                          Drama|War  
737                                         Comedy|War